In [3]:
import json




In [4]:
# Load the JSON file
with open(r"D:\radipaem\all_papers_metadata.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # Read JSON content



In [5]:
# Get the first 50 records
first_50_records = data[:1]

# Print or process the records
for record in first_50_records:
    print(record)

[{'tag': 'title', 'text': 'Transitional care for rheumatic conditions in Europe: current clinical practice and available resources'}, {'tag': 'orgName', 'text': 'German Leibniz Association'}, {'tag': 'orgName', 'text': 'EULAR/ PReS'}, {'tag': 'publisher', 'text': 'Springer Science and Business Media LLC'}, {'tag': 'p', 'text': 'Copyright Springer Science and Business Media LLC'}, {'tag': 'date', 'text': '2017-06-09'}, {'tag': 'forename', 'text': 'Daniel'}, {'tag': 'surname', 'text': 'Clemente'}, {'tag': 'orgName', 'text': 'Paediatric Rheumatology Unit'}, {'tag': 'orgName', 'text': 'Hospital Infantil Universitario "Niño Jesús"'}, {'tag': 'settlement', 'text': 'Madrid'}, {'tag': 'country', 'text': 'Spain'}, {'tag': 'forename', 'text': 'Leticia'}, {'tag': 'surname', 'text': 'Leon'}, {'tag': 'orgName', 'text': 'IDISSC'}, {'tag': 'orgName', 'text': 'Hospital Clínico San Carlos; Health Sciences'}, {'tag': 'orgName', 'text': 'Universidad Camilo José Cela'}, {'tag': 'settlement', 'text': 'Madr

## NER Medical

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [20]:
#tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
#model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")
#

In [2]:
ner_path = r'D:\radipaem\biomedical-ner-all'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(ner_path)
model = AutoModelForTokenClassification.from_pretrained(ner_path)

In [4]:
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple") # pass device=0 if using gpu
#'''
pipe("""Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels over a prolonged period.

The primary cause of diabetes is the body's inability to produce enough insulin or effectively use the insulin it produces.

Common symptoms include frequent urination, increased thirst, unexplained weight loss, and fatigue.

Older adults and individuals with a family history of diabetes are more susceptible to this condition.

Medications like Metformin are commonly prescribed to manage blood sugar levels in diabetic patients.
""")
#'''


Device set to use cpu


[{'entity_group': 'Disease_disorder',
  'score': np.float32(0.9993943),
  'word': 'diabetes mellitus',
  'start': 0,
  'end': 17},
 {'entity_group': 'Disease_disorder',
  'score': np.float32(0.9769399),
  'word': 'metabolic disorder',
  'start': 31,
  'end': 49},
 {'entity_group': 'Lab_value',
  'score': np.float32(0.97860795),
  'word': 'high',
  'start': 67,
  'end': 71},
 {'entity_group': 'Diagnostic_procedure',
  'score': np.float32(0.9784419),
  'word': 'sugar',
  'start': 78,
  'end': 83},
 {'entity_group': 'Medication',
  'score': np.float32(0.8636727),
  'word': 'insulin',
  'start': 189,
  'end': 196},
 {'entity_group': 'Sign_symptom',
  'score': np.float32(0.7684654),
  'word': 'ur',
  'start': 275,
  'end': 277},
 {'entity_group': 'Subject',
  'score': np.float32(0.69303346),
  'word': 'older adults',
  'start': 343,
  'end': 355},
 {'entity_group': 'Family_history',
  'score': np.float32(0.7693746),
  'word': 'family history of diabetes',
  'start': 379,
  'end': 405},
 {'e

In [5]:
# Input text
text = """Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels over a prolonged period.
The primary cause of diabetes is the body's inability to produce enough insulin or effectively use the insulin it produces.
Common symptoms include frequent urination, increased thirst, unexplained weight loss, and fatigue.
Older adults and individuals with a family history of diabetes are more susceptible to this condition.
Medications like Metformin are commonly prescribed to manage blood sugar levels in diabetic patients."""

# Get NER output
ner_output = pipe(text)

# Extract unique entity groups
unique_entity_groups = list(set(entry['entity_group'] for entry in ner_output))

print(unique_entity_groups)

['Sign_symptom', 'Subject', 'Lab_value', 'Diagnostic_procedure', 'Disease_disorder', 'Family_history', 'Medication']


In [6]:
#!pip install torch transformers
#!pip install accelerate 
#!pip install scipy
#!pip install --upgrade transformers torch numpy scikit-learn


## Triplex model

In [7]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
def triplextract(model, tokenizer, text, entity_types, predicates):

    input_format = """Perform Named Entity Recognition (NER) and extract knowledge graph triplets from the text. NER identifies named entities of given entity types, and triple extraction identifies relationships between entities using specified predicates.
      
        **Entity Types:**
        {entity_types}
        
        **Predicates:**
        {predicates}
        
        **Text:**
        {text}
        """

    message = input_format.format(
                entity_types = json.dumps({"entity_types": entity_types}),
                predicates = json.dumps({"predicates": predicates}),
                text = text)

    messages = [{'role': 'user', 'content': message}]
    #input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt = True, return_tensors="pt").to("cuda")
    #output = tokenizer.decode(model.generate(input_ids=input_ids, max_length=2048)[0], skip_special_tokens=True)
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt = True, return_tensors="pt")
    output = tokenizer.decode(model.generate(input_ids=input_ids, max_length=2048)[0], skip_special_tokens=True)
    return output


In [9]:
path = r'D:\radipaem\Triplex'

In [10]:
#model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True).to('cuda').eval()
#tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

###### medical text

In [11]:
entity_types = unique_entity_groups #["DISEASE", "SYMPTOM", "MEDICINE", "REASON"]
#predicates = ["CAUSES", "AFFECTS", "TREATED_BY", "HAS_SYMPTOM"]
predicates = [
    # Disease-Related Relations
    "causes", "associated_with", "risk_factor_for", "complication_of", 
    "symptom_of", "diagnosed_by",
    
    # Treatment-Related Relations
    "treated_with", "prescribed_for", "contraindicated_for", "side_effect_of",
    
    # Drug-Related Relations
    "interacts_with", "metabolized_by", "has_dosage",
    
    # Anatomy & Physiology Relations
    "affects", "located_in", "part_of",
    
    # General Clinical Relations
    "has_test_result", "suggests", "caused_by"
]
text = """Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels over a prolonged period.

The primary cause of diabetes is the body's inability to produce enough insulin or effectively use the insulin it produces.

Common symptoms include frequent urination, increased thirst, unexplained weight loss, and fatigue.

Older adults and individuals with a family history of diabetes are more susceptible to this condition.

Medications like Metformin are commonly prescribed to manage blood sugar levels in diabetic patients.
"""
prediction = triplextract(model, tokenizer, text, entity_types, predicates)
print(prediction)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Perform Named Entity Recognition (NER) and extract knowledge graph triplets from the text. NER identifies named entities of given entity types, and triple extraction identifies relationships between entities using specified predicates.
      
        **Entity Types:**
        {"entity_types": ["Sign_symptom", "Subject", "Lab_value", "Diagnostic_procedure", "Disease_disorder", "Family_history", "Medication"]}
        
        **Predicates:**
        {"predicates": ["causes", "associated_with", "risk_factor_for", "complication_of", "symptom_of", "diagnosed_by", "treated_with", "prescribed_for", "contraindicated_for", "side_effect_of", "interacts_with", "metabolized_by", "has_dosage", "affects", "located_in", "part_of", "has_test_result", "suggests", "caused_by"]}
        
        **Text:**
        Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels over a prolonged period.

The primary cause of diabetes is the body's inability to produce enough insu

In [33]:
unique_entity_groups

['Subject',
 'Disease_disorder',
 'Family_history',
 'Diagnostic_procedure',
 'Lab_value',
 'Sign_symptom',
 'Medication']

In [10]:
entity_types = [ "LOCATION", "POSITION", "DATE", "CITY", "COUNTRY", "NUMBER" ]
predicates = [ "POPULATION", "AREA" ]
text = """
San Francisco,[24] officially the City and County of San Francisco, is a commercial, financial, and cultural center in Northern California. 

With a population of 808,437 residents as of 2022, San Francisco is the fourth most populous city in the U.S. state of California behind Los Angeles, San Diego, and San Jose.
"""



In [11]:
prediction = triplextract(model, tokenizer, text, entity_types, predicates)
print(prediction)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Perform Named Entity Recognition (NER) and extract knowledge graph triplets from the text. NER identifies named entities of given entity types, and triple extraction identifies relationships between entities using specified predicates.
      
        **Entity Types:**
        {"entity_types": ["LOCATION", "POSITION", "DATE", "CITY", "COUNTRY", "NUMBER"]}
        
        **Predicates:**
        {"predicates": ["POPULATION", "AREA"]}
        
        **Text:**
        
San Francisco,[24] officially the City and County of San Francisco, is a commercial, financial, and cultural center in Northern California. 

With a population of 808,437 residents as of 2022, San Francisco is the fourth most populous city in the U.S. state of California behind Los Angeles, San Diego, and San Jose.

         
  ```json
{
    "entities_and_triples": [
        "[1], CITY:San Francisco",
        "[2], LOCATION:City and County of San Francisco",
        "[3], LOCATION:Northern California",
        "[4], NUMBE

In [12]:
entity_types = ["CASE", "LAWYER", "DATE"]
predicates = ["VERDICT", "CHARGES"]
text = """
In the landmark case of Roe v. Wade (1973), lawyer Sarah Weddington successfully argued before the US Supreme Court, leading to a verdict that protected women's reproductive rights.
"""
prediction = triplextract(model, tokenizer, text, entity_types, predicates)
print(prediction)

Perform Named Entity Recognition (NER) and extract knowledge graph triplets from the text. NER identifies named entities of given entity types, and triple extraction identifies relationships between entities using specified predicates.
      
        **Entity Types:**
        {"entity_types": ["CASE", "LAWYER", "DATE"]}
        
        **Predicates:**
        {"predicates": ["VERDICT", "CHARGES"]}
        
        **Text:**
        
In the landmark case of Roe v. Wade (1973), lawyer Sarah Weddington successfully argued before the US Supreme Court, leading to a verdict that protected women's reproductive rights.

         
  ```json
{
    "entities_and_triples": [
        "[1], CASE:Roe v. Wade",
        "[2], DATE:1973",
        "[3], LAWYER:Sarah Weddington",
        "[4], LAWYER:lawyer",
        "[5], LAWYER:US Supreme Court",
        "[6], LAWYER:US Supreme Court",
        "[1] VERDICT [7]",
        "[7], CASE:verdict that protected women's reproductive rights",
        "[3] CHARGE

In [2]:
text = '''
Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels over a prolonged period. The primary cause of diabetes is the body's inability to produce enough insulin or effectively use the insulin it produces. Common symptoms include frequent urination, increased thirst, unexplained weight loss, and fatigue. Older adults and individuals with a family history of diabetes are more susceptible to this condition. Medications like Metformin are commonly prescribed to manage blood sugar levels in diabetic patients.
'''
print(text)


Diabetes mellitus is a chronic metabolic disorder characterized by high blood sugar levels over a prolonged period. The primary cause of diabetes is the body's inability to produce enough insulin or effectively use the insulin it produces. Common symptoms include frequent urination, increased thirst, unexplained weight loss, and fatigue. Older adults and individuals with a family history of diabetes are more susceptible to this condition. Medications like Metformin are commonly prescribed to manage blood sugar levels in diabetic patients.



In [2]:
import torch
print(torch.cuda.is_available())

False


In [3]:
import numpy as np
print(np.__version__)

2.2.2
